In [1]:
print ("hello")

hello


In [1]:
import os

In [2]:
%pwd

'd:\\Develop\\Pipline\\Face_recognitions\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Develop\\Pipline\\Face_recognitions'

Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url:str
    zip_data_File:Path
    unzip_dir:Path


src configaration management

In [8]:
from face_recognitions.constants import *
from face_recognitions.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_YAML_PATH,
                 params_filepath=PARAMS_YAML_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            zip_data_File=config.zip_data_File,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
        


Data ingestion pipeline

In [46]:

import os
import urllib.request as request
import zipfile
from face_recognitions import logger
from face_recognitions.utils.common import get_size

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.zip_data_File):
            filename,headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.zip_data_File
            )
            logger.info(f"{filename} download ! with flowwing info : \n {headers}")
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.zip_data_File))}')


    def extract_file(self):
        '''
        Zip_file_path:str
        Extract the zip file into data directory
        Function returns None
        
        '''
        unzip_path=self.config.unzip_dir

        logger.info(f"Zip file extracting to {unzip_path}")
        
        # os.makedirs(unzip_path, exist_ok=True)
        create_directories([unzip_path])
        with zipfile.ZipFile(self.config.zip_data_File,'r')as zip_ref:
            zip_ref.extractall(unzip_path)

        logger.info(f"Zip file extracted to {unzip_path}")
        



main

In [47]:
try:
    config=ConfigurationManager()
    dat_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=dat_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_file()
except Exception as e:
    raise e

[2024-02-26 16:19:50,633: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-26 16:19:50,638: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-26 16:19:50,640: INFO: common: created directory at: artifacts]
[2024-02-26 16:19:50,642: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-26 16:23:17,307: INFO: 726149500: artifacts/data_ingestion/data.zip download ! with flowwing info : 
 X-GUploader-UploadID: ABPtcPrjUYS1thXdFDBJucYTpmMvMqZQDcRb_92UMCZmI-PgPDVpX5Ei-ujgvFXBrEh4F85qvkA
Content-Type: application/octet-stream
Content-Security-Policy: sandbox
Content-Security-Policy: default-src 'none'
Content-Security-Policy: frame-ancestors 'none'
X-Content-Security-Policy: sandbox
Cross-Origin-Opener-Policy: same-origin
Cross-Origin-Embedder-Policy: require-corp
Cross-Origin-Resource-Policy: same-site
X-Content-Type-Options: nosniff
Content-Disposition: attachment; filename="CelebA_HQ_facial_identity_dataset.zip"
Access-Control-Allo